<a href="https://colab.research.google.com/github/camara94/computer_vision_lab/blob/main/ecommerce_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import time
import json
import numpy as np
import PIL.Image, PIL.ImageFont, PIL.ImageDraw


from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import ResNet50

from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

In [ ]:
drive.mount("/content/drive/")

In [ ]:
!ls ./drive/MyDrive/data_4

In [ ]:
base_dir = './drive/MyDrive/data_4'

In [ ]:
# n_classes = len(liste_dossier)
# n_classes

# classes = []

# for c in liste_dossier:
#     classes.append(' '.join(c.title().split('_')))

# for classe in classes:
#     with open("ecommerce_computer_vision/classes.txt", "a+", encoding="UTF-8") as fichier:
#         fichier.write(classe + "\n")

In [ ]:
BATCH_SIZE = 32 
classes = []
liste_dossier = os.listdir(base_dir+'/train')
for c in liste_dossier:
    classes.append(' '.join(c.title().split('_')))
classes[:4]

In [ ]:
# Define our example directories and files

train_dir = os.path.join( base_dir, 'train')
test_dir = os.path.join( base_dir, 'test')
validation_dir = os.path.join( base_dir, 'val')



# Ajouter nos paramètres d'augmentation de données à ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                   # data augmentation
                                   rotation_range = 50,
                                   width_shift_range = 0.3,
                                   height_shift_range = 0.3,
                                   shear_range = 0.3,
                                   zoom_range = 0.3,
                                   horizontal_flip = True)

# Notez que les données de validation ne doivent pas être augmentées !
validation_datagen = ImageDataGenerator( rescale = 1.0/255 )
test_datagen = ImageDataGenerator( rescale = 1.0/255 )

# Images d'entraînement de flux par lots de 20 à l'aide du générateur train_datagen
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 895,
                                                    class_mode = 'categorical', 
                                                    target_size = (56, 56))  

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 111,
                                                          class_mode  = 'categorical', 
                                                          target_size = (56, 56))

# Flow validation images in batches of 20 using test_datagen generator
test_generator =  test_datagen.flow_from_directory( test_dir,
                                                          batch_size  = 115,
                                                          class_mode  = 'categorical', 
                                                          target_size = (56, 56))

In [ ]:
len(test_generator)

In [ ]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims

In [ ]:
'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor

In [ ]:
'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    # Ajoutez une couche entièrement connectée avec 1 024 unités cachées et l'activation ReLU
    x = Dense(1024, activation=keras.layers.ReLU())(x)

    # Ajouter un taux d'abandon de 0,2
    x = Dropout(0.1)(x) 

    # Ajoutez une couche entièrement connectée avec 1 024 unités cachées et l'activation ReLU
    x = Dense(512, activation=keras.layers.ReLU())(x)

    # Ajouter un taux d'abandon de 0,2
    x = Dropout(0.1)(x)

    # # Ajoutez une couche entièrement connectée avec 1 024 unités cachées et l'activation ReLU
    x = Dense(256, activation=keras.layers.ReLU())(x)

    # # Ajouter un taux d'abandon de 0,2
    x = Dropout(0.1)(x) 

    x = tf.keras.layers.Dense(len(classes), activation="softmax", name="classification")(x)
    return x

In [ ]:
'''
Since input image size is (56 x 56), first upsample the image by factor of (4x4) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(4,4))(inputs)

    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

In [ ]:
'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(56,56,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [ ]:
model = define_compile_model()

model.summary()

In [ ]:
# Définissez une classe de rappel qui arrête l'entraînement une fois que la précision atteint 99.9 %
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.99):
            print("\nAtteint 98 % de précision, donc annulation de l'entraînement !")
            self.model.stop_training = True

In [ ]:
training_images = []
training_labels = []
training_label = []
training_images.append( np.array(train_generator[0][0]) )
training_label = [np.argmax(i) for i in train_generator[0][1]]
training_labels.append( np.array(training_label).reshape(np.array(training_label).shape[0], 1) )

In [ ]:
validation_images = []
validation_labels = []
validation_label = []
validation_images.append( np.array(validation_generator[0][0]) )
validation_label = [np.argmax(i) for i in validation_generator[0][1]]
validation_labels.append( np.array(validation_label).reshape(np.array(validation_label).shape[0], 1) )

In [ ]:
training_labels = np.array(training_labels)
training_images = np.array(training_images)

validation_labels = np.array(validation_labels)
validation_images = np.array(validation_images)
    

In [ ]:
if training_images.ndim == 5:
    training_images = training_images[0]
training_images.shape

In [ ]:
if validation_images.ndim == 5:
    validation_images = validation_images[0]
validation_images.shape

In [ ]:
# training_label = np.array(training_images)
# training_label = training_label.reshape(training_label.shape[0], 1)
training_labels = training_labels[0]
training_labels.shape 

In [ ]:
# validation_labels = np.array(validation_labels)
# validation_labels = training_label.reshape(validation_labels.shape[0], 1)
validation_labels = validation_labels[0]
validation_labels.shape

In [ ]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims

In [ ]:
# train_X = preprocess_image_input(training_images)
# valid_X = preprocess_image_input(validation_images)

In [ ]:
# this will take around 20 minutes to complete
EPOCHS = 10
history = model.fit(
    training_images, 
    training_labels, 
    epochs=EPOCHS, 
    validation_data = (validation_images, validation_labels), 
    batch_size=64
)